<a href="https://colab.research.google.com/github/jajapuramshivasai/Q24_2/blob/main/1D_wave_HHl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit==0.37.2

# implementing HHL algorithm#

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit.providers.aer import AerSimulator

In [ ]:
#tridiagonal example
matrix = np.array([[2, -1, 0, 0], [-1, 2, -1, 0], [0, -1,  2, -1],[0, 0, -1, 2]])
vector = [0.707, 0.707, 0.707, 0.707]

naive_hhl_solution = HHL().solve(matrix, vector)

In [ ]:
print(naive_hhl_solution.state)

      ┌─────────────┐┌──────┐        ┌─────────┐
q4_0: ┤0            ├┤5     ├────────┤5        ├
      │  circuit-87 ││      │        │         │
q4_1: ┤1            ├┤6     ├────────┤6        ├
      └─────────────┘│      │┌──────┐│         │
q5_0: ───────────────┤0     ├┤4     ├┤0        ├
                     │      ││      ││         │
q5_1: ───────────────┤1 QPE ├┤3     ├┤1 QPE_dg ├
                     │      ││      ││         │
q5_2: ───────────────┤2     ├┤2     ├┤2        ├
                     │      ││  1/x ││         │
q5_3: ───────────────┤3     ├┤1     ├┤3        ├
                     │      ││      ││         │
q5_4: ───────────────┤4     ├┤0     ├┤4        ├
                     └──────┘│      │└─────────┘
  q6: ───────────────────────┤5     ├───────────
                             └──────┘           


In [ ]:
from qiskit.quantum_info import Statevector
def get_solution_vector(solution,a,b):
    """
    Extracts and normalizes simulated state vector
    from LinearSolverResult.
    """
    solution_vector = Statevector(solution.state).data[a:b].real
    norm = solution.euclidean_norm
    return norm * solution_vector / np.linalg.norm(solution_vector)

# print('full naive solution vector:', get_solution_vector(naive_hhl_solution))

In [ ]:
def solve_linear_system_using_hll(matrix, vector):
  l = len(vector)
  naive_hhl_solution = HHL().solve(matrix, vector) #epsilon=0.0001

  q = naive_hhl_solution.state.num_qubits-1
  a = 2**(q)
  b = 2**(q) + l

  return get_solution_vector(naive_hhl_solution,a,b)


In [ ]:
classical_solution = np.linalg.solve(matrix, vector/np.linalg.norm(vector))

In [ ]:
print('classical solution:', classical_solution)

classical solution: [1.  1.5 1.5 1. ]


In [ ]:
q = solve_linear_system_using_hll(matrix, vector)
print('quantum solution:',q)

quantum solution: [0.99892872 1.50066209 1.50066209 0.99892872]


# 1d wave simulation using implicit finite difference method #

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import rc
from matplotlib.animation import FuncAnimation
rc('animation', html='jshtml')

In [ ]:
# simulation Parameters
L = 1.0  # Length of the domain
T = 2  # Total time
c = 1.0  # Wave speed = tau/ro
Nx = 7  # Number of spatial grid points
Nt = 24  # Number of time steps
dx = L / Nx
dt = T / Nt
r = c * dt / dx



x = np.linspace(0, L, Nx+1) # X space

k = (np.pi/L)
w = 1*np.pi/T
t0 = -dt/2
t1 = dt/2
u0 = 2*np.sin(k*x)*np.sin(w*t0)
u1 = 2*np.sin(k*x)*np.sin(w*t1)


# Create tridiagonal matrix for the implicit scheme
"""
for example for n = 5
[1+2*r**2   -1*r**2     ,0          ,0          ,0        ]   |0       |
[-1*r**2    ,1+2*r**2   ,-1*r**2    ,0          ,0        ]   |U(t+1,1)| ___
[0          ,-1*r**2    ,1+2*r**2   ,-1*r**2    ,0        ] X |U(t+1,2)| ___.   2*[U(t,x)] - [U(t-1,x)]
[0          ,0          ,-1*r**2    ,1+2*r**2   ,-1*r**2  ]   |U(t+1,3)|
[0          ,0          ,0          ,-1*r**2    ,1+2*r**2 ]   |0       |

"""
A = np.zeros((Nx+1, Nx+1))
A[0, 0] = 1+2*r**2
A[0, 1] = -1*r**2
A[Nx, Nx] = 1+2*r**2
A[Nx, Nx-1] = -1*r**2
for i in range(1, Nx):
    A[i, i-1] = -1* r**2
    A[i, i] = 1 + 2*r**2
    A[i, i+1] = -1 * r**2

print(A)

# Time stepping
u = np.zeros((Nt+1, Nx+1))
u[0, :] = u0
u[1, :] = u1

for n in range(1, Nt):
    b = 2*u[n] - u[n-1]
    b[0] = 0  # Boundary conditions
    b[Nx] = 0


# Solve for the next time step using classical algorithm

    u[n+1, :] = np.linalg.solve(A, b)

# solve for the next time step using quantum algorithm

    # u[n+1, :] = solve_linear_system_using_hll(A, b)


[[ 1.68055556 -0.34027778  0.          0.          0.          0.
   0.          0.        ]
 [-0.34027778  1.68055556 -0.34027778  0.          0.          0.
   0.          0.        ]
 [ 0.         -0.34027778  1.68055556 -0.34027778  0.          0.
   0.          0.        ]
 [ 0.          0.         -0.34027778  1.68055556 -0.34027778  0.
   0.          0.        ]
 [ 0.          0.          0.         -0.34027778  1.68055556 -0.34027778
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.34027778  1.68055556
  -0.34027778  0.        ]
 [ 0.          0.          0.          0.          0.         -0.34027778
   1.68055556 -0.34027778]
 [ 0.          0.          0.          0.          0.          0.
  -0.34027778  1.68055556]]


In [ ]:


# Animation
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(xlim=(0, L), ylim=(-1.2, 1.2))
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    line.set_data(x, u[i, :])
    return line,

anim_C = FuncAnimation(fig, animate, init_func=init, frames=Nt+1, interval=1000, blit=True)


In [ ]:
anim_C

In [ ]:
anim_C.save('classical_wave_animation.gif', writer='pillow')

In [ ]:
# simulation Parameters
L = 1.0  # Length of the domain
T = 2  # Total time
c = 1.0  # Wave speed = tau/ro
Nx = 7  # Number of spatial grid points
Nt = 24  # Number of time steps
dx = L / Nx
dt = T / Nt
r = c * dt / dx



x = np.linspace(0, L, Nx+1) # X space

k = (np.pi/L)
w = 1*np.pi/T
t0 = -dt/2
t1 = dt/2
u0 = np.sin(k*x)*np.sin(w*t0)
u1 = np.sin(k*x)*np.sin(w*t1)

def sin_pulse(x, t , n):
    return np.sin(k*x)*np.sin(w*t)

# Create tridiagonal matrix for the implicit scheme
"""
for example for n = 5
[1+2*r**2   -1*r**2     ,0          ,0          ,0        ]   |0       |
[-1*r**2    ,1+2*r**2   ,-1*r**2    ,0          ,0        ]   |U(t+1,1)| ___
[0          ,-1*r**2    ,1+2*r**2   ,-1*r**2    ,0        ] X |U(t+1,2)| ___.   2*[U(t,x)] - [U(t-1,x)]
[0          ,0          ,-1*r**2    ,1+2*r**2   ,-1*r**2  ]   |U(t+1,3)|
[0          ,0          ,0          ,-1*r**2    ,1+2*r**2 ]   |0       |

"""
A = np.zeros((Nx+1, Nx+1))
A[0, 0] = 1+2*r**2
A[0, 1] = -1*r**2
A[Nx, Nx] = 1+2*r**2
A[Nx, Nx-1] = -1*r**2
for i in range(1, Nx):
    A[i, i-1] = -1* r**2
    A[i, i] = 1 + 2*r**2
    A[i, i+1] = -1 * r**2

print(A)

# Time stepping
u_q = np.zeros((Nt+1, Nx+1))
u_q[0, :] = u0
u_q[1, :] = u1

for n in range(1, Nt):
    b = 2*u[n] - u[n-1]
    b[0] = 0  # Boundary conditions
    b[Nx] = 0


# Solve for the next time step using classical algorithm

    # u[n+1, :] = np.linalg.solve(A, b)

# solve for the next time step using quantum algorithm

    u_q[n+1, :] = solve_linear_system_using_hll(A, b)

[[ 1.68055556 -0.34027778  0.          0.          0.          0.
   0.          0.        ]
 [-0.34027778  1.68055556 -0.34027778  0.          0.          0.
   0.          0.        ]
 [ 0.         -0.34027778  1.68055556 -0.34027778  0.          0.
   0.          0.        ]
 [ 0.          0.         -0.34027778  1.68055556 -0.34027778  0.
   0.          0.        ]
 [ 0.          0.          0.         -0.34027778  1.68055556 -0.34027778
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.34027778  1.68055556
  -0.34027778  0.        ]
 [ 0.          0.          0.          0.          0.         -0.34027778
   1.68055556 -0.34027778]
 [ 0.          0.          0.          0.          0.          0.
  -0.34027778  1.68055556]]


In [ ]:
# Animation
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(xlim=(0, L), ylim=(-1.2, 1.2))
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    line.set_data(x, u[i, :])
    return line,

anim_Q = FuncAnimation(fig, animate, init_func=init, frames=Nt+1, interval=1000, blit=True)

In [ ]:
anim_Q

In [ ]:
anim_Q.save('quantum_wave_animation.gif', writer='pillow')